In [19]:
from google.colab import drive
drive.mount('/content/drive')
# %cd /content/drive/MyDrive/Colab Notebooks/Foodie_CLIP

Mounted at /content/drive


In [3]:
!git clone --branch yusa https://github.com/yussaaa/Foodie_CLIP.git

Cloning into 'Foodie_CLIP'...
remote: Enumerating objects: 84, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 84 (delta 31), reused 63 (delta 21), pack-reused 0
Receiving objects: 100% (84/84), 36.26 KiB | 12.09 MiB/s, done.
Resolving deltas: 100% (31/31), done.


In [13]:
!pwd

/content


In [14]:
%cd Foodie_CLIP/

/content/Foodie_CLIP


In [5]:
!mkdir data/

In [17]:
!cp -r /content/drive/MyDrive/Colab\ Notebooks/FoodieCLIP_data/train600x600/ Foodie_CLIP/data/

In [20]:
!cp -r /content/drive/MyDrive/Colab\ Notebooks/FoodieCLIP_data/val600x600/ Foodie_CLIP/data/


^C


In [14]:
!cp -r /content/drive/MyDrive/Colab\ Notebooks/FoodieCLIP_data/class_names.csv Foodie_CLIP/data/

In [11]:
!pip install transformers[torch]
!pip install torcheval
!pip install torch
!pip install accelerate

  Using cached accelerate-0.31.0-py3-none-any.whl (309 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 5.7 MB/s eta 0:00:00


In [15]:
from foodie_clip.foodieclip import FoodieClip, ClIPLoss
from trainer.utils import CLIPDataCollator, CLIPDataset
from trainer.evaluate import compute_accuracy
from foodie_clip.foodieclip import FoodieClip, ClIPLoss
from trainer.utils import CLIPDataset, CLIPDataCollator
from trainer.clip_trainer import CLIPTrainer
from transformers import (
    TrainingArguments,
    AutoProcessor,
    CLIPConfig,
    EarlyStoppingCallback,
)

In [16]:
import json

# Read the config.json file
with open('config.json') as f:
    config = json.load(f)

# Access the config parameters
DATA_DIRECTORY = config["DATA_DIRECTORY"]

print("DATA_DIRECTORY: ", DATA_DIRECTORY)

DATA_DIRECTORY:  data/


In [17]:
import torch

# Check if MPS is available and set the device
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print("Training on device: ", device)

Training on device:  cuda


In [18]:
# loading model
foodieclip_config = CLIPConfig.from_pretrained("openai/clip-vit-base-patch32")
foodieclip = FoodieClip(foodieclip_config, projection_dim=128).to(device)
# loading dataset
train_dataset = CLIPDataset(train=True,path=DATA_DIRECTORY, class_max_datapoints=200)
print(len(train_dataset))
eval_dataset = CLIPDataset(train=False, path=DATA_DIRECTORY, class_max_datapoints=100)
# set data collator
processor = AutoProcessor.from_pretrained("openai/clip-vit-base-patch32")
data_collator = CLIPDataCollator(processor=processor)
# set up training args
training_args = TrainingArguments(
    output_dir="output/",
    per_device_train_batch_size=500,
    evaluation_strategy="steps",
    logging_steps=1,
    num_train_epochs=2,
    report_to=None,
    eval_steps=5,
    fp16=False,
    remove_unused_columns=False,
    load_best_model_at_end=True,
)
trainer = CLIPTrainer(
    model=foodieclip,
    training_args=training_args,
    data_collator=data_collator,
    max_length=128,
    compute_metrics=compute_accuracy,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    Loss_FN=ClIPLoss,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

47177


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

...

Step,Training Loss,Validation Loss,caption match accuracy
5,5.836100,1.094805,[0.948204]
10,5.778800,1.075416,[0.95128775]
15,5.802000,1.076899,[0.949289]
20,5.768900,1.075556,[0.94677633]
25,5.771900,1.075570,[0.94877505]
30,5.769700,1.075520,[0.9472903]
35,5.790300,1.075517,[0.9479756]
40,5.796600,1.075438,[0.94894636]
45,5.787100,1.075449,[0.95123065]
50,5.780900,1.075391,[0.9525441]


Trainer is attempting to log a value of "[0.948204]" of type <class 'numpy.ndarray'> for key "eval/ caption match accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.95128775]" of type <class 'numpy.ndarray'> for key "eval/ caption match accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.949289]" of type <class 'numpy.ndarray'> for key "eval/ caption match accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.94677633]" of type <class 'numpy.ndarray'> for key "eval/ caption match accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.94877505]" of type <

Step,Training Loss,Validation Loss,caption match accuracy
5,5.836100,1.094805,[0.948204]
10,5.778800,1.075416,[0.95128775]
15,5.802000,1.076899,[0.949289]
20,5.768900,1.075556,[0.94677633]
25,5.771900,1.075570,[0.94877505]
30,5.769700,1.075520,[0.9472903]
35,5.790300,1.075517,[0.9479756]
40,5.796600,1.075438,[0.94894636]
45,5.787100,1.075449,[0.95123065]
50,5.780900,1.075391,[0.9525441]


Trainer is attempting to log a value of "[0.95300096]" of type <class 'numpy.ndarray'> for key "eval/ caption match accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.9540289]" of type <class 'numpy.ndarray'> for key "eval/ caption match accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.9561418]" of type <class 'numpy.ndarray'> for key "eval/ caption match accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.9541431]" of type <class 'numpy.ndarray'> for key "eval/ caption match accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.95677]" of type <cl

TrainOutput(global_step=190, training_loss=5.6324566389385025, metrics={'train_runtime': 8990.9036, 'train_samples_per_second': 10.494, 'train_steps_per_second': 0.021, 'total_flos': 0.0, 'train_loss': 5.6324566389385025, 'epoch': 2.0})

In [20]:
dir(trainer)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_activate_neftune',
 '_add_sm_patterns_to_gitignore',
 '_created_lr_scheduler',
 '_deactivate_neftune',
 '_finish_current_push',
 '_fsdp_qlora_plugin_updates',
 '_gather_and_numpify',
 '_get_collator_with_removed_columns',
 '_get_eval_sampler',
 '_get_learning_rate',
 '_get_output_dir',
 '_get_train_sampler',
 '_globalstep_last_logged',
 '_hp_search_setup',
 '_inner_training_loop',
 '_issue_warnings_after_load',
 '_load_best_model',
 '_load_callback_state',
 '_load_from_checkpoint',
 '_load_optimizer_and_scheduler',
 '_load_rng_state',
 '_loggers_initialized',
 '_maybe_log_save_evaluate',
 '_memory_tracker',
 '_move_mode

In [21]:
!mkdir model

In [24]:
trainer.save_model()

TypeError: FoodieClipPreTrain.save_pretrained() got an unexpected keyword argument 'state_dict'

In [27]:
bestmodel = trainer.model

In [28]:
bestmodel.save_pretrained("model/bestmodel")

In [ ]:
bestmodel.predict("data/val600x600/00")

In [29]:
test_model = FoodieClip.from_pretrained("model/bestmodel")


In [30]:
test_model


FoodieClip(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 512)
      (position_embedding): Embedding(77, 512)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=512, out_features=2048, bias=True)
            (fc2): Linear(in_features=2048, out_features=512, bias=True)
          )
          (layer_norm2): LayerNorm((512,), eps=1e-05

## Download data from Kaggle

In [3]:
!mkdir .kaggle

mkdir: cannot create directory ‘.kaggle’: File exists


In [5]:
!mv kaggle.json ~/.kaggle

In [6]:
!chmod 600 ~/.kaggle/kaggle.json

chmod: cannot access '/root/.kaggle/kaggle.json': Not a directory


In [7]:
!ls -a


.  ..  .config	.kaggle  sample_data


In [8]:
!kaggle datasets download -d zachaluza/cnfood-241

Dataset URL: https://www.kaggle.com/datasets/zachaluza/cnfood-241
License(s): other
100% 9.75G/9.75G [10:33<00:00, 16.9MB/s]
100% 9.75G/9.75G [10:33<00:00, 16.5MB/s]


In [33]:
!mv ../.kaggle/ ~/


In [4]:
%cd Foodie_CLIP/


/content/Foodie_CLIP


In [5]:
!pwd


/content/Foodie_CLIP


In [38]:
!ls

config.json  foodie_clip       README.md	 services  trainer
data	     foodieclip.ipynb  requirements.txt  src	   train.py


In [40]:
!ls ~/

In [52]:
!kaggle datasets list -s 'cnfood'

ref                   title       size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------  ----------  ----  -------------------  -------------  ---------  ---------------  
zachaluza/cnfood-241  CNFOOD-241  10GB  2023-06-17 14:15:55            150          6  0.9411765        


In [9]:
!unzip cnfood-241.zip -d ./Foodie_CLIP/data/

Streaming output truncated to the last 5000 lines.
  inflating: ./Foodie_CLIP/data/val600x600/159/000119.jpg  
  inflating: ./Foodie_CLIP/data/val600x600/159/000120.jpg  
  inflating: ./Foodie_CLIP/data/val600x600/159/000121.jpg  
  inflating: ./Foodie_CLIP/data/val600x600/159/000122.jpg  
  inflating: ./Foodie_CLIP/data/val600x600/159/000123.jpg  
  inflating: ./Foodie_CLIP/data/val600x600/159/000124.jpg  
  inflating: ./Foodie_CLIP/data/val600x600/159/000125.jpg  
  inflating: ./Foodie_CLIP/data/val600x600/159/000126.jpg  
  inflating: ./Foodie_CLIP/data/val600x600/159/000127.jpg  
  inflating: ./Foodie_CLIP/data/val600x600/159/000128.jpg  
  inflating: ./Foodie_CLIP/data/val600x600/159/000129.jpg  
  inflating: ./Foodie_CLIP/data/val600x600/159/000130.jpg  
  inflating: ./Foodie_CLIP/data/val600x600/160/000000.jpg  
  inflating: ./Foodie_CLIP/data/val600x600/160/000001.jpg  
  inflating: ./Foodie_CLIP/data/val600x600/160/000002.jpg  
  inflating: ./Foodie_CLIP/data/val600x600/160/00